In [ ]:
import time

import sdcommon
import txt2img

def run(**kwargs):
    ml = txt2img.ML()
    params = txt2img.Params(**kwargs)
    start = time.time()
    img, _ = ml.run(params) #, local_files_only=False)
    print("Took %.1fs" % (time.time()-start))
    print("Saved as %s.png" % sdcommon.save(ml, params, img[0]))
    return img[0]

prompt = "a squirrel cat hybrid"
prompt = "Old man making clockwork children toys in his workshop. Happy place. Holidays."
img = run(prompt=prompt, steps=25)
img